## Code

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
!pip install xlrd
!pip install openpyxl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.5/96.5 kB 21.6 MB/s eta 0:00:00

[notice] A new release of pip available: 22.2.1 -> 22.3.1
[notice] To update, run: pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.1/242.1 kB 25.4 MB/s eta 0:00:00

[notice] A new release of pip available: 22.2.1 -> 22.3.1
[notice] To update, run: pip install --upgrade pip


## Purchasing Power Parity (PPP)

In [2]:
ppp = pd.read_excel('ppp.xls', 'Data', header = 3) # main data
ppp_metadata = pd.read_excel('ppp.xls', 'Metadata - Countries') # incomegroup / region

ppp = ppp.set_index('Country Code') # country code as index
ppp_metadata = ppp_metadata.set_index('Country Code') # country code as index
ppp = ppp[['Country Name', 'Indicator Name', '2005', '2006', '2007', 
           '2008', '2009', '2010', '2011', '2012', '2013', '2014', 
           '2015', '2016', '2017', '2018', '2019', '2020', '2021']] # years since 2005
ppp['Average PPP'] = ppp.mean(axis=1) # averaging data points
ppp = pd.merge(ppp, ppp_metadata[['Region', 'IncomeGroup']], how='outer', 
               left_index=True, right_index=True) # merging dfs to get region and income group
ppp = ppp.dropna() # dropping na values

ppp['Development Group'] = None

ppp.loc[ppp['Average PPP'] < 3975.735294, 'Development Group'] = 'Low Development' 
ppp.loc[ppp['Average PPP'] >= 3975.735294, 'Development Group'] = 'Middle Low Development' 
ppp.loc[ppp['Average PPP'] >= 10340.588235, 'Development Group'] = 'Middle High Development' 
ppp.loc[ppp['Average PPP'] >= 22743.235294, 'Development Group'] = 'High Development' 

ppp.head()

,Country Name,Indicator Name,2005,2006,2007,2008,2009,2010,2011,2012,...,2016,2017,2018,2019,2020,2021,Average PPP,Region,IncomeGroup,Development Group
Country Code,,,,,,,,,,,,,,,,,,,,,
AGO,Angola,"GNI per capita, PPP (current international $)",4290.0,4710.0,5330.0,5560.0,5820.0,5950.0,6130.0,6810.0,...,6730.0,6860.0,6600.0,6360.0,5930.0,6060.0,6171.176471,Sub-Saharan Africa,Lower middle income,Middle Low Development
ALB,Albania,"GNI per capita, PPP (current international $)",5980.0,6760.0,7490.0,8270.0,8670.0,9530.0,10210.0,10450.0,...,12280.0,12800.0,13480.0,13820.0,13410.0,15430.0,10734.705882,Europe & Central Asia,Upper middle income,Middle High Development
ARG,Argentina,"GNI per capita, PPP (current international $)",12030.0,14470.0,16160.0,16990.0,15830.0,17450.0,18770.0,19180.0,...,19860.0,22990.0,22460.0,22090.0,20230.0,23150.0,18845.294118,Latin America & Caribbean,Upper middle income,Middle High Development
ARM,Armenia,"GNI per capita, PPP (current international $)",5690.0,6720.0,7970.0,8800.0,7530.0,7880.0,8310.0,9740.0,...,10950.0,12540.0,13230.0,14420.0,13100.0,14220.0,10141.176471,Europe & Central Asia,Upper middle income,Middle Low Development
ATG,Antigua and Barbuda,"GNI per capita, PPP (current international $)",19450.0,22270.0,24620.0,24570.0,21500.0,20120.0,19720.0,18950.0,...,18570.0,18780.0,20470.0,21340.0,17910.0,19610.0,20111.176471,Latin America & Caribbean,High income,Middle High Development


In [3]:
print('low income')
print(ppp.loc[ppp['Development Group'] == 'Low Development', 'Country Name'])
print('\nmiddle low income')
print(ppp.loc[ppp['Development Group'] == 'Middle Low Development', 'Country Name'])
print('\nmiddle high income')
print(ppp.loc[ppp['Development Group'] == 'Middle High Development', 'Country Name'])
print('\nhigh income')
print(ppp.loc[ppp['Development Group'] == 'High Development', 'Country Name'])

low income
Country Code
BDI                     Burundi
BEN                       Benin
BFA                Burkina Faso
BGD                  Bangladesh
CAF    Central African Republic
CIV               Cote d'Ivoire
CMR                    Cameroon
COD            Congo, Dem. Rep.
COM                     Comoros
ETH                    Ethiopia
FSM       Micronesia, Fed. Sts.
GIN                      Guinea
GMB                 Gambia, The
GNB               Guinea-Bissau
HTI                       Haiti
KEN                       Kenya
KGZ             Kyrgyz Republic
KHM                    Cambodia
LBR                     Liberia
LSO                     Lesotho
MDG                  Madagascar
MLI                        Mali
MMR                     Myanmar
MOZ                  Mozambique
MWI                      Malawi
NER                       Niger
NPL                       Nepal
PNG            Papua New Guinea
RWA                      Rwanda
SDN                       Sudan
SEN             

## Choropleth

Are there any regions where all of the countries have a similar average PPP, whether that be high or low? Returns could be different across different regions and this trend could be interesting to look at for further investigation.

In [4]:
ppp = ppp.reset_index() # reseting index to get country code information

# building choropleth using plotly
ppp_fig = go.Figure(data=go.Choropleth(
    locations = ppp['Country Code'],
    z = ppp['Average PPP'], # color shading = income group
    text = ppp['Country Name'],
    colorscale = 'Greens',
    autocolorscale=False,
    marker_line_color='darkgray',
    marker_line_width=0.5,
    # colorbar_tickprefix = '%',
    colorbar_title = 'PPP',
))

ppp_fig.update_layout(
    title_text='Purchasing Power Parity of Countries', # adding title
    geo=dict(
        showframe=False,
        showcoastlines=False,
    )
)

ppp_fig.show()

Seeing a lot of low PPP averages in sub-saharan Africa and a lot of high PPP averages in North America and Europe. It could be interesting to compare returns in Europe vs. returns in sub-saharan Africa.

## Stacked Bar Chart

The World Bank provides income groups that could be used to group development. We already split income by percentiles of PPP. Let's see how they compare to the income groups.

In [5]:
ppp_bar = ppp
ppp_bar['count'] = 1
ppp_bar = ppp_bar.sort_values('Region')
ppp_bar_chart = px.bar(ppp_bar, x="Development Group", y="count", color="IncomeGroup", 
                       hover_data = ['Country Name'], title="Development Distribution by Income Group")

ppp_bar_chart.update_layout(xaxis={'categoryorder':'array', 'categoryarray':['Low Development',
                                   'Middle Low Development','Middle High Development',
                                   'High Development']})
ppp_bar_chart.show()

It looks like the income groups span across multiple percentiles and aren't consistent for lower middle income and upper middle income. Sticking with our determined PPP percentiles will be better.

## Line Chart

Another concern is that outliers on either edge will influence the results of a complex analysis.

In [6]:
ppp_line = ppp
ppp_line = ppp_bar.sort_values('Average PPP')

ppp_line_chart = px.line(ppp_line, x="Country Name", y="Average PPP", hover_data = ['Country Name'], 
                         title='Average PPP per country')

ppp_line_chart.add_annotation(x='Russian Federation', y=22755.882352941175, text="Start High Development", 
                              showarrow=True, arrowhead=1)

ppp_line_chart.add_annotation(x='Paraguay', y=10370.0, text="Start Middle High Development", 
                              showarrow=True, arrowhead=1)

ppp_line_chart.add_annotation(x='Ghana', y=4121.764705882353, text="Start Middle Low Development", 
                              showarrow=True, arrowhead=1)

ppp_line_chart.update_layout(xaxis={'visible': False, 'showticklabels': False})


ppp_line_chart.show()

We see a large disparity between the lowest "High Development" (Russia) country and the highest "High Development" country (Qatar). For our purposes, this should pose any serious issue, as we are looking at countries on an individual basis, but if you are wanting to use this data set for something more advanced, you may consider removing outliers or creating a new group for the highest PPP countries.

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=6e0cd533-862b-451a-ac5c-be06b38b3cd3' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>